In [1]:
import pandas as pd
import json
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
import torch
from datasets import load_metric
from sklearn.model_selection import train_test_split
import numpy as np

# Load the JSON datasets
with open(r"C:\Users\KIIT\Desktop\CSV files\EXIST 2024 Tweets Dataset\dev\EXIST2024_dev.json", encoding='utf-8') as f:
    dev_data = json.load(f)
with open(r"C:\Users\KIIT\Desktop\CSV files\EXIST 2024 Tweets Dataset\training\EXIST2024_training.json", encoding='utf-8') as f:
    train_data = json.load(f)

# Convert the JSON data to DataFrames
def json_to_dataframe(json_data):
    records = []
    for key, value in json_data.items():
        record = {
            'id': value['id_EXIST'],
            'lang': value['lang'],
            'tweet': value['tweet'],
            'labels_task3': value['labels_task3'],
            'split': value['split']
        }
        records.append(record)
    return pd.DataFrame(records)

train_df = json_to_dataframe(train_data)
dev_df = json_to_dataframe(dev_data)

# Combine the training and development data
combined_df = pd.concat([train_df, dev_df])

# Split the data into train and validation sets
train_df, val_df = train_test_split(combined_df, test_size=0.2, random_state=42)

# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Encode the data
def encode_data(tokenizer, df, max_length=128):
    texts = df['tweet'].tolist()
    return tokenizer(texts, padding='max_length', truncation=True, max_length=max_length, return_tensors='pt')

train_encodings = encode_data(tokenizer, train_df)
val_encodings = encode_data(tokenizer, val_df)

# Possible labels for task3
possible_labels_task3 = ["IDEOLOGICAL-INEQUALITY", "STEREOTYPING-DOMINANCE", "OBJECTIFICATION", 
                         "SEXUAL-VIOLENCE", "MISOGYNY-NON-SEXUAL-VIOLENCE", "-", "UNKNOWN"]

# Encode labels for task3
def encode_labels_task3(labels_list, possible_labels):
    encoded_labels = []
    for labels in labels_list:
        encoded_label = [0] * len(possible_labels)
        for label_set in labels:
            for label in label_set:
                if label in possible_labels:
                    encoded_label[possible_labels.index(label)] = 1
        encoded_labels.append(encoded_label)
    return encoded_labels

train_labels_task3 = encode_labels_task3(train_df['labels_task3'].tolist(), possible_labels_task3)
val_labels_task3 = encode_labels_task3(val_df['labels_task3'].tolist(), possible_labels_task3)

# Define the Dataset class
class HinglishDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)

# Create dataset objects
train_dataset = HinglishDataset(train_encodings, train_labels_task3)
val_dataset = HinglishDataset(val_encodings, val_labels_task3)

# Load the metrics
accuracy_metric = load_metric("accuracy")
precision_metric = load_metric("precision")
recall_metric = load_metric("recall")
f1_metric = load_metric("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    sigmoid_logits = torch.sigmoid(torch.tensor(logits))
    predictions = (sigmoid_logits > 0.5).int().numpy()
    labels = np.array(labels)
    
    # Flatten the arrays
    predictions = predictions.flatten()
    labels = labels.flatten()
    
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    precision = precision_metric.compute(predictions=predictions, references=labels, average='micro')
    recall = recall_metric.compute(predictions=predictions, references=labels, average='micro')
    f1 = f1_metric.compute(predictions=predictions, references=labels, average='micro')
    
    return {
        "accuracy": accuracy['accuracy'],
        "precision": precision['precision'],
        "recall": recall['recall'],
        "f1": f1['f1']
    }

# Initialize the model for multi-label classification
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=len(possible_labels_task3))

# Training arguments
training_args = TrainingArguments(
    output_dir='./results_task3',
    num_train_epochs=8,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs_task3',
    logging_steps=10,
    evaluation_strategy="epoch"
)

# Data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Evaluate on the validation set
val_results = trainer.evaluate(eval_dataset=val_dataset)
print(f"Validation Results for labels_task3: {val_results}")

# Separate block for testing accuracy
test_results = trainer.predict(val_dataset)
print(f"Testing Accuracy: {test_results.metrics['test_accuracy']}")
print(f"Testing Precision: {test_results.metrics['test_precision']}")
print(f"Testing Recall: {test_results.metrics['test_recall']}")
print(f"Testing F1 Score: {test_results.metrics['test_f1']}")

C:\Users\KIIT\anaconda3\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\KIIT\AppData\Local\Temp\ipykernel_21528\3315561522.py:88: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  accuracy_metric = load_metric("accuracy")
C:\Users\KIIT\anaconda3\lib\site-packages\datasets\load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Pas

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.498800,0.470938,0.764986,0.764986,0.764986,0.764986
2,0.441400,0.453556,0.777100,0.777100,0.777100,0.777100
3,0.427900,0.451830,0.784368,0.784368,0.784368,0.784368
4,0.341100,0.473258,0.785086,0.785086,0.785086,0.785086
5,0.258600,0.508563,0.780689,0.780689,0.780689,0.780689
6,0.227800,0.538382,0.779433,0.779433,0.779433,0.779433
7,0.183200,0.569058,0.782215,0.782215,0.782215,0.782215
8,0.172600,0.584888,0.780689,0.780689,0.780689,0.780689


Validation Results for labels_task3: {'eval_loss': 0.5848882794380188, 'eval_accuracy': 0.780689160086145, 'eval_precision': 0.780689160086145, 'eval_recall': 0.780689160086145, 'eval_f1': 0.780689160086145, 'eval_runtime': 303.1493, 'eval_samples_per_second': 5.252, 'eval_steps_per_second': 0.33, 'epoch': 8.0}
Testing Accuracy: 0.780689160086145
Testing Precision: 0.780689160086145
Testing Recall: 0.780689160086145
Testing F1 Score: 0.780689160086145


In [2]:
# Save the model and tokenizer
model.save_pretrained('./saved_model_BERTtask3')
tokenizer.save_pretrained('./saved_tokenizer_BERTtask3')

('./saved_tokenizer_BERTtask3\\tokenizer_config.json',
 './saved_tokenizer_BERTtask3\\special_tokens_map.json',
 './saved_tokenizer_BERTtask3\\vocab.txt',
 './saved_tokenizer_BERTtask3\\added_tokens.json')